# Listeners must have arguments "event" and "metadata" with proper types

In [1]:
from morpheus.common.types.event_sourcing.EventMetadata import EventMetadata
from morpheus.project.domain.events.ProjectEvents import ProjectCreatedEvent
from morpheus.common.infrastructure.event_sourcing.EventPublisher import EventPublisher, listen_to


class Invalid:
  pass


class ProjectorInvalid0:
  @listen_to(ProjectCreatedEvent)
  def on_project_created(self):
    print('Created project')


class ProjectorInvalid1:
  @listen_to(ProjectCreatedEvent)
  def on_project_created(self, invalid: ProjectCreatedEvent, metadata: EventMetadata):
    print('Created project')


class ProjectorInvalid2:
  @listen_to(ProjectCreatedEvent)
  def on_project_created(self, event: ProjectCreatedEvent, invalid: EventMetadata):
    print('Created project')


class ProjectorInvalid3:
  @listen_to(ProjectCreatedEvent)
  def on_project_created(self, event: Invalid, metadata: EventMetadata):
    print('Created project')


class ProjectorInvalid4:
  @listen_to(ProjectCreatedEvent)
  def on_project_created(self, event: ProjectCreatedEvent, metadata: Invalid):
    print('Created project')


event_publisher = EventPublisher()
try:
  projector = ProjectorInvalid0()
  event_publisher.register(projector)
except Exception as e:
  print(e)

try:
  projector = ProjectorInvalid1()
  event_publisher.register(projector)
except Exception as e:
  print(e)

try:
  projector = ProjectorInvalid2()
  event_publisher.register(projector)
except Exception as e:
  print(e)

try:
  projector = ProjectorInvalid3()
  event_publisher.register(projector)
except Exception as e:
  print(e)

try:
  projector = ProjectorInvalid4()
  event_publisher.register(projector)
except Exception as e:
  print(e)

Event listener <bound method ProjectorInvalid0.on_project_created of <__main__.ProjectorInvalid0 object at 0x133d70110>> must have an argument called "event" that should by type hinted with a subclass of EventBase
Event listener <bound method ProjectorInvalid1.on_project_created of <__main__.ProjectorInvalid1 object at 0x123d0fb30>> must have an argument called "event" that should by type hinted with a subclass of EventBase
Event listener <bound method ProjectorInvalid2.on_project_created of <__main__.ProjectorInvalid2 object at 0x104058e30>> must have an argument called "metadata" that should by type hinted with a subclass of EventMetadata
Event listener <bound method ProjectorInvalid3.on_project_created of <__main__.ProjectorInvalid3 object at 0x12292cce0>> must have an argument called "event" that should by type hinted with a subclass of EventBase
Event listener <bound method ProjectorInvalid4.on_project_created of <__main__.ProjectorInvalid4 object at 0x104058e30>> must have an arg

# Cannot register same listener twice

In [2]:
class Projector:
  @listen_to(ProjectCreatedEvent)
  def on_project_created(self, event: ProjectCreatedEvent, metadata: EventMetadata):
    print(f'Created project {event.get_project_id()} with metadata {metadata}')


try:
  event_publisher = EventPublisher()
  projector = Projector()
  event_publisher.register(projector)
  event_publisher.register(projector)
except Exception as e:
  print(e)

Event handler already registered


# Create projects

In [3]:
from morpheus.project.infrastructure.persistence.ProjectSummaryRepository import project_summary_repository
from morpheus.project.infrastructure.event_sourcing.ProjectEventStore import project_event_store
from morpheus.project.types.User import UserId
from morpheus.project.types.Project import ProjectId, Name, Description, Tags
from morpheus.project.application.write.Project import CreateProjectCommand, CreateProjectCommandHandler

for i in range(10):
  create_project_command = CreateProjectCommand(
    project_id=ProjectId.new(),
    name=Name.from_str(f'Project {i}'),
    description=Description.from_str(f'Description {i}'),
    tags=Tags.from_list([f'tag{i}', f'tag{i + 1}']),
    user_id=UserId.new()
  )
  CreateProjectCommandHandler.handle(create_project_command)

event_envelopes = project_event_store.get_all_events_ordered_by_version()
print(f'Found {len(event_envelopes)} events')
for event_envelope in event_envelopes:
  print(f'"{event_envelope.get_event().get_event_name().to_str()}" occurred at {event_envelope.event.get_occurred_at().to_iso_with_timezone()}')

project_summaries = project_summary_repository.find_all()
print(f'Found {len(project_summaries)} project summaries')
for project_summary in project_summaries:
  print(f'Project {project_summary.project_id.to_str()}: {project_summary.project_name.to_str()}')

Found 83 events
"Project Created" occurred at 2024-04-22T11:12:44.096283+02:00
"Project Created" occurred at 2024-04-22T11:20:25.259302+02:00
"Project Created" occurred at 2024-04-22T12:25:00.703853+02:00
"Project Created" occurred at 2024-04-22T12:25:30.987298+02:00
"Project Created" occurred at 2024-04-22T12:25:43.781157+02:00
"Project Created" occurred at 2024-04-22T22:34:54.712343+02:00
"Project Created" occurred at 2024-04-22T22:34:54.727891+02:00
"Project Created" occurred at 2024-04-22T22:34:54.733633+02:00
"Project Created" occurred at 2024-04-22T22:34:54.740777+02:00
"Project Created" occurred at 2024-04-22T22:34:54.745596+02:00
"Project Created" occurred at 2024-04-22T22:34:54.750669+02:00
"Project Created" occurred at 2024-04-22T22:34:54.754796+02:00
"Project Created" occurred at 2024-04-22T22:34:54.758468+02:00
"Project Created" occurred at 2024-04-22T22:34:54.761277+02:00
"Project Created" occurred at 2024-04-22T22:34:54.764123+02:00
"Project Created" occurred at 2024-04-2

# Reproject project summaries

In [4]:
from morpheus.project.presentation.cli.ProjectionCliCommands import ReprojectProjectSummariesCliCommand

ReprojectProjectSummariesCliCommand.run()

Reprojecting project summaries
------------------------------

Found 83 events
Reset projection
Successfully reprojected project summaries


In [5]:
project_summaries = project_summary_repository.find_all()
print(f'Found {len(project_summaries)} project summaries')
for project_summary in project_summaries:
  print(f'Project {project_summary.project_id.to_str()}: {project_summary.project_name.to_str()}')

Found 35 project summaries
Project 83195f32-5944-407b-9ff6-1b70888a9d64: Rio Primero
Project 8b0eb583-8ef4-4925-8bd5-c2876bb2f821: Rio Primero
Project 14fd5e74-6bb0-4c37-acf9-7ea95340dc45: Rio Primero
Project 5c8e2cab-4f0d-414a-ae04-5b8770c4d093: Rio Primero
Project f672cab7-7de8-4f52-9771-75ec3f3bd17f: Rio Primero
Project fc01b724-20c7-47f9-bc77-38e9cd6b5c8f: Project 0
Project 6150ef53-da68-48ff-8ba5-d34c365acf8d: Project 1
Project 1d7d64a1-f752-490d-b0b1-2f0862458439: Project 2
Project 5e12a88c-1d7e-4de4-9fbc-e5c512882e05: Project 3
Project bdb79e1f-f326-4df2-a9f6-c20a77a6dc9b: Project 4
Project 71f41e97-aeff-4953-a9bd-58b5342e90e2: Project 5
Project 80bd1bad-7dd2-4483-b728-7aa5a6b4e742: Project 6
Project bbccb0b5-89a2-4f6e-9612-e26643021967: Project 7
Project c2482077-b026-472e-a426-9ce61ab7e3a8: Project 8
Project 60818f6b-a69d-4e13-8ee9-3fb9aa021508: Project 9
Project 6bf12650-19a1-4fa4-828e-98390e574e62: Project 0
Project e90f8571-4466-4123-86dc-ecf5418b499f: Project 1
Project 130